In [13]:
%pip install matplotlib torch


  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
    --------------------------------------- 3.9/241.4 MB 26.0 MB/s eta 0:00:10
   - -------------------------------------- 8.7/241.4 MB 26.8 MB/s eta 0:00:09
   -- ------------------------------------- 14.7/241.4 MB 24.9 MB/s eta 0:00:10
   --- ------------------------------------ 20.7/241.4 MB 25.7 MB/s eta 0:00:09
   ---- ----------------------------------- 24.9/241.4 MB 23.9 MB/s eta 0:00:10
   ----- ---------------------------------- 30.4/241.4 MB 24.4 MB/s eta 0:00:09
   ----- ---------------------------------- 34.6/241.4 MB 23.4 MB/s eta 0:00:09
   ------ --------------------------------- 38.8/241.4 MB 23.5 MB/s eta 0:00:09
   ------- --------------------------

In [15]:
%pip install scipy

   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   --- ------------------------------------ 3.4/41.3 MB 18.3 MB/s eta 0:00:03
   -------- ------------------------------- 9.2/41.3 MB 22.0 MB/s eta 0:00:02
   ----------- ---------------------------- 12.3/41.3 MB 19.8 MB/s eta 0:00:02
   ---------------- ----------------------- 17.3/41.3 MB 20.2 MB/s eta 0:00:02
   -------------------- ------------------- 21.0/41.3 MB 19.8 MB/s eta 0:00:02
   ------------------------ --------------- 25.4/41.3 MB 20.1 MB/s eta 0:00:01
   ---------------------------- ----------- 29.1/41.3 MB 19.2 MB/s eta 0:00:01
   ------------------------------- -------- 32.5/41.3 MB 19.1 MB/s eta 0:00:01
   ------------------------------------ --- 37.2/41.3 MB 19.4 MB/s eta 0:00:01
   ---------------------------------------  41.2/41.3 MB 19.2 MB/s eta 0:00:01
   ---------------------------------------- 41.3/41.3 MB 18.4 MB/s  0:00

In [ ]:
# Parses the rollout data from demo.py and saves them as numpy arrays of states, actions, and pot handle positions (the conditional vector data)

import numpy as np
import csv
import matplotlib.pyplot as plt
import pickle as pkl
import torch
from scipy.spatial.transform import Rotation as R
from transform_utils import quat_to_rot6d, rotvec_to_rot6d, rot6d_to_quat
from image_encoder import VisionTransformerEncoder


hello
torch


In [ ]:
# Initialize ViT encoder for image processing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit_encoder = VisionTransformerEncoder(latent_dim=128)
vit_encoder.to(device).eval()

VisionTransformerEncoder(
  (patch_embed): ImagePatchAndEmbed(
    (projection): Linear(in_features=768, out_features=768, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): MultiHeadAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (projection): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=768, out_features=3072, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.1, inplace=False)
        (3): Linear(in_features=3072, out_features=768, bias=True)
        (4): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_fe

In [ ]:
# with open("rollouts_pot/rollout_seed0_mode2.pkl", "rb") as f:
#     rollout = pkl.load(f)
#     obs = rollout["observations"]
#     actions = np.array(rollout["actions"])
#     robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
#     robot0_eef_quat = np.array([o["robot0_eef_quat"] for o in obs])
#     robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
#     robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
#     robot1_eef_quat = np.array([o["robot1_eef_quat"] for o in obs])
#     robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

#     repeats_needed = 250 - actions.shape[0]

#     repeated_last = np.tile(actions[-1], (repeats_needed, 1))
#     actions = np.vstack([actions, repeated_last])

#     repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
#     robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
#     state = robot0_eef_pos

#     repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
#     robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
#     robot0_eef_rotvec = R.from_quat(robot0_eef_quat).as_rotvec()
#     state = np.hstack([state, robot0_eef_rotvec])


#     repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
#     robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
#     robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
#     state = np.hstack([state, robot0_gripper_pos])

#     repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
#     robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
#     state = np.hstack([state, robot1_eef_pos])

#     repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
#     robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
#     robot1_eef_rotvec = R.from_quat(robot1_eef_quat).as_rotvec()
#     state = np.hstack([state, robot1_eef_rotvec])

#     repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
#     robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
#     robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
#     state = np.hstack([state, robot1_gripper_pos])

# print(np.shape(state))
# print(np.shape(actions))

In [20]:
expert_states_list = []
expert_actions_list = []
pot_start_list = []
pot_states_list1 = []
pot_states_list2 = []
image_latents_list0 = []
image_latents_list1 = []
for i in [2, 3]:
    for j in range(10):   
        with open("rollouts/newslower/rollout_seed%s_mode%s.pkl" % (j*10, i), "rb") as f:
            rollout = pkl.load(f)
            obs = rollout["observations"]
            actions = np.array(rollout["actions"])
            print(np.shape(actions))
            pot1 = np.array(rollout["pot_states1"])
            pot2 = np.array(rollout["pot_states2"])
            pot = np.array(rollout["pot_start"])

            pot_start_list.append(np.concatenate((pot[0], pot[1])))

            if 'camera0_obs' in rollout and 'camera1_obs' in rollout:
                camera0_latents = []
                camera1_latents = []
                for frame_idx in range(len(rollout['camera0_obs'])):
                    img0 = torch.from_numpy(rollout['camera0_obs'][frame_idx]).permute(2,0,1).unsqueeze(0).float().to(device)
                    img1 = torch.from_numpy(rollout['camera1_obs'][frame_idx]).permute(2,0,1).unsqueeze(0).float().to(device)
                    with torch.no_grad():
                        camera0_latents.append(vit_encoder(img0).cpu().numpy().squeeze())
                        camera1_latents.append(vit_encoder(img1).cpu().numpy().squeeze())
                camera0_latents = np.array(camera0_latents)
                camera1_latents = np.array(camera1_latents)
            else:
                camera0_latents = np.zeros((len(actions), 128))
                camera1_latents = np.zeros((len(actions), 128))

            robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
            robot0_eef_quat = np.array([o["robot0_eef_quat_site"] for o in obs])
            robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
            robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
            robot1_eef_quat = np.array([o["robot1_eef_quat_site"] for o in obs])
            robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

            repeats_needed = 700 - actions.shape[0]

            repeated_last = np.tile(actions[-1], (repeats_needed, 1))
            actions = np.vstack([actions, repeated_last])

            repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
            robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
            state = robot0_eef_pos

            repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
            robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
            robot0_eef_rotvec = R.from_quat(robot0_eef_quat).as_rotvec()
            state = np.hstack([state, robot0_eef_rotvec])


            repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
            robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
            robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
            state = np.hstack([state, robot0_gripper_pos])

            repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
            robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
            state = np.hstack([state, robot1_eef_pos])

            repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
            robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
            robot1_eef_rotvec = R.from_quat(robot1_eef_quat).as_rotvec()
            state = np.hstack([state, robot1_eef_rotvec])

            repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
            robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
            robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
            state = np.hstack([state, robot1_gripper_pos])

            repeated_last = np.tile(pot1[-1], (repeats_needed, 1))
            pot1 = np.vstack([pot1, repeated_last])
            repeated_last = np.tile(pot2[-1], (repeats_needed, 1))
            pot2 = np.vstack([pot2, repeated_last])
            pot_states_list1.append(pot1)
            pot_states_list2.append(pot2)

            repeated_last = np.tile(camera0_latents[-1], (repeats_needed, 1))
            camera0_latents = np.vstack([camera0_latents, repeated_last])
            repeated_last = np.tile(camera1_latents[-1], (repeats_needed, 1))
            camera1_latents = np.vstack([camera1_latents, repeated_last])
            image_latents_list0.append(camera0_latents)
            image_latents_list1.append(camera1_latents)

            expert_states_list.append(state)
            expert_actions_list.append(actions)

expert_states_rotvec = np.stack(expert_states_list, axis=0)
expert_actions_rotvec = np.stack(expert_actions_list, axis=0)
pot_states_rotvec1 = np.stack(pot_states_list1, axis=0)
pot_states_rotvec2 = np.stack(pot_states_list2, axis=0)
pot_start_rotvec = np.stack(pot_start_list, axis=0)
image_latents_rotvec0 = np.stack(image_latents_list0, axis=0)
image_latents_rotvec1 = np.stack(image_latents_list1, axis=0)

FileNotFoundError: [Errno 2] No such file or directory: 'rollouts/newslower/rollout_seed0_mode2.pkl'

In [ ]:
print(np.shape(expert_states_rotvec))
print(np.shape(expert_actions_rotvec))
print(np.shape(pot_states_rotvec1))
print(np.shape(pot_states_rotvec2))
print(np.shape(pot_start_rotvec))
print(np.shape(image_latents_rotvec0))
print(np.shape(image_latents_rotvec1))

(20, 700, 14)
(20, 700, 14)
(20, 700, 3)
(20, 700, 3)
(20, 6)


In [ ]:
np.save("data/expert_states_newslower_20.npy", expert_states_rotvec)
np.save("data/expert_actions_newslower_20.npy", expert_actions_rotvec)
np.save("data/pot_states1_newslower_20.npy", pot_states_rotvec1)
np.save("data/pot_states2_newslower_20.npy", pot_states_rotvec2)
np.save("data/pot_start_newslower_20.npy", pot_start_rotvec)
np.save("data/arm1_images_latents.npy", image_latents_rotvec0)
np.save("data/arm2_images_latents.npy", image_latents_rotvec1)


In [ ]:
# with open("rollouts/rollout_seed0_mode2.pkl", "rb") as f:
#     rollout = pkl.load(f)
#     obs = rollout["observations"]
#     actions = np.array(rollout["actions"])

#     pos0 = actions[:,:3]
#     rotvec0 = actions[:,3:6]
#     gripper0 = actions[:,6]
#     pos1 = actions[:,7:10]
#     rotvec1 = actions[:,10:13]
#     gripper1 = actions[:,13]

#     rot6d_list0 = []
#     for rv in rotvec0:
#         rot6d_list0.append(rotvec_to_rot6d(rv))
#     rot6d0 = np.array(rot6d_list0)

#     rot6d_list1 = []
#     for rv in rotvec1:
#         rot6d_list1.append(rotvec_to_rot6d(rv))
#     rot6d1 = np.array(rot6d_list1)

#     actions = np.concatenate((pos0, rot6d0, gripper0.reshape(-1, 1), pos1, rot6d1, gripper1.reshape(-1, 1)), axis=1)

#     robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
#     robot0_eef_quat = np.array([o["robot0_eef_quat"] for o in obs])
#     robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
#     robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
#     robot1_eef_quat = np.array([o["robot1_eef_quat"] for o in obs])
#     robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

#     repeats_needed = 250 - actions.shape[0]

#     repeated_last = np.tile(actions[-1], (repeats_needed, 1))
#     actions = np.vstack([actions, repeated_last])

#     repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
#     robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
#     state = robot0_eef_pos

#     repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
#     robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
#     eef_rot6d0 = []
#     for q in robot0_eef_quat:
#         eef_rot6d0.append(quat_to_rot6d(q))
#     robot0_eef_rot6d = np.array(eef_rot6d0)
#     state = np.hstack([state, robot0_eef_rot6d])


#     repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
#     robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
#     robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
#     state = np.hstack([state, robot0_gripper_pos])

#     repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
#     robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
#     state = np.hstack([state, robot1_eef_pos])

#     repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
#     robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
#     eef_rot6d1 = []
#     for q in robot1_eef_quat:
#         eef_rot6d1.append(quat_to_rot6d(q))
#     robot1_eef_rot6d = np.array(eef_rot6d1)
#     state = np.hstack([state, robot1_eef_rot6d])

#     repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
#     robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
#     robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
#     state = np.hstack([state, robot1_gripper_pos])
    
# print(np.shape(state))
# print(np.shape(actions))

In [15]:
expert_states_list = []
expert_actions_list = []
pot_states_list = []
for i in [2, 3]:
    for j in range(10):   
        with open("rollouts_grippause/rollout_seed%s_mode%s.pkl" % (j*10, i), "rb") as f:
            rollout = pkl.load(f)
            obs = rollout["observations"]
            actions = np.array(rollout["actions"])
            pot = np.array(rollout["pot_pos"])

            pot_states_list.append(np.concatenate((pot[0], pot[1])))

            pos0 = actions[:,:3]
            rotvec0 = actions[:,3:6]
            gripper0 = actions[:,6]
            pos1 = actions[:,7:10]
            rotvec1 = actions[:,10:13]
            gripper1 = actions[:,13]

            rot6d_list0 = []
            for rv in rotvec0:
                rot6d_list0.append(rotvec_to_rot6d(rv))
            rot6d0 = np.array(rot6d_list0)

            rot6d_list1 = []
            for rv in rotvec1:
                rot6d_list1.append(rotvec_to_rot6d(rv))
            rot6d1 = np.array(rot6d_list1)

            actions = np.concatenate((pos0, rot6d0, gripper0.reshape(-1, 1), pos1, rot6d1, gripper1.reshape(-1, 1)), axis=1)

            robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
            robot0_eef_quat = np.array([o["robot0_eef_quat_site"] for o in obs])
            robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
            robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
            robot1_eef_quat = np.array([o["robot1_eef_quat_site"] for o in obs])
            robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

            repeats_needed = 400 - actions.shape[0]

            repeated_last = np.tile(actions[-1], (repeats_needed, 1))
            actions = np.vstack([actions, repeated_last])

            repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
            robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
            state = robot0_eef_pos

            repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
            robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
            eef_rot6d0 = []
            for q in robot0_eef_quat:
                eef_rot6d0.append(quat_to_rot6d(q))
            robot0_eef_rot6d = np.array(eef_rot6d0)
            state = np.hstack([state, robot0_eef_rot6d])


            repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
            robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
            robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
            state = np.hstack([state, robot0_gripper_pos])

            repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
            robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
            state = np.hstack([state, robot1_eef_pos])

            repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
            robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
            eef_rot6d1 = []
            for q in robot1_eef_quat:
                eef_rot6d1.append(quat_to_rot6d(q))
            robot1_eef_rot6d = np.array(eef_rot6d1)
            state = np.hstack([state, robot1_eef_rot6d])

            repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
            robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
            robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
            state = np.hstack([state, robot1_gripper_pos])

            expert_states_list.append(state)
            expert_actions_list.append(actions)

expert_states_rot6d = np.stack(expert_states_list, axis=0)
expert_actions_rot6d = np.stack(expert_actions_list, axis=0)
pot_states_rot6d = np.stack(pot_states_list, axis=0)

In [16]:
print(np.shape(expert_states_rot6d))
print(np.shape(expert_actions_rot6d))
print(np.shape(pot_states_rot6d))

(20, 400, 20)
(20, 400, 20)
(20, 6)


In [17]:
np.save("data/expert_states_rot6d_site_grippause_20.npy", expert_states_rot6d)
np.save("data/expert_actions_rot6d_site_grippause_20.npy", expert_actions_rot6d)
np.save("data/pot_states_rot6d_site_grippause_20.npy", pot_states_rot6d)